In [1]:
import pandas as pd
import trollcamp

config = trollcamp.load_config()

is_test = config['is_test']
file_path = config['file_path']
batch_name = config['batch_name']
token_key = config['token_key']
db_config = config['db_config']

In [2]:
import glob

def get_league_file_path():
    if is_test == True:
        return glob.glob("%s/%s/leagues/*.json" % (file_path, batch_name))
    else:
        return trollcamp.AWS_files_in_dir("%s/leagues/" % batch_name, config)

In [3]:
league_file_path = get_league_file_path()
#league_file_path

In [4]:
players_df = pd.DataFrame(columns=['account_id'])
players_call_api_df = pd.DataFrame(columns=['summoner_id', 'summoner_name'])
players_error_df = pd.DataFrame(columns=['summoner_id', 'summoner_name'])

In [5]:
cnx = trollcamp.db_connect(db_config)

In [6]:
for path in league_file_path:
    data = trollcamp.read_json(path, config, True)
    
    tier = data['tier']

    for player in data['entries']:
        summoner_id = player["playerOrTeamId"]
        summoner_name = player["playerOrTeamName"]
        rank = player["rank"]

        (result, account_id) = trollcamp.db_check_player(cnx, summoner_id)

        if result == True:
            players_df = players_df.append({'account_id': account_id}, ignore_index=True)
        else:
            players_call_api_df = players_call_api_df.append({'summoner_id': summoner_id, 'summoner_name': summoner_name}, ignore_index=True)
                
#players_call_api_df

In [ ]:
# 테스트로 3개만 진행한다. 실제로는 temp_count 변수 관련 모두 삭제하고 실행할 것!

#temp_count = 0
for key, value in players_call_api_df.iterrows():
    summoner_id = value["summoner_id"]
    summoner_name = value["summoner_name"]
    
    (result, account_id) = trollcamp.call_api_summoners(summoner_id, token_key)
    
    if result == True:
        players_df = players_df.append({'account_id': account_id}, ignore_index=True)
        trollcamp.db_insert_player(cnx, account_id, summoner_id, summoner_name)
    else:
        players_error_df = players_error_df.append({'summoner_id': summoner_id, 'summoner_name': summoner_name}, ignore_index=True)
        
    #temp_count += 1
    #if temp_count >= 3:
        #break

print("Api call is completed.") 

In [8]:
trollcamp.db_close(cnx)

In [10]:
def get_max_rows(df_length, split_count):
    if (df_length % split_count) > 0:
        return (df_length // split_count) + 1
    else:
        return (df_length // split_count) 

def get_split_dataframes(df):
    max_rows = get_max_rows(len(df), config['chunk_count'])
    split_dfs = []
    while len(df) > max_rows:
        top = df[:max_rows]
        split_dfs.append(top)
        df = df[max_rows:]
    else:
        split_dfs.append(df)
    return split_dfs

In [ ]:
#trollcamp.df_to_csv(players_df, "%s/players.csv" % (batch_name), config)

split_dfs = get_split_dataframes(players_df)
for idx, split_df in enumerate(split_dfs):
    trollcamp.df_to_csv(split_df, "%s/players/players-%d.csv" % (batch_name, (idx+1)), config)